In [47]:
import numpy as np
import scipy as sp

# Constants
g = 9.81
fs = 1000      # Hz, sample frequency
t = np.arange(0, 100, 1/fs)

# Accelerometer specs (XSens MTI-680G RTK)
b = 15e-6*g    # bias
S_a = 60e-6*g  # noise density

# Filter constants
order = 2
f_c = 0.2
overlap = 1.5
zeta = 0.4
wc = 0.14*2*np.pi
wc_w = 2*fs*np.tan(wc/(2*fs)) 

# Complementary filter numerator/denominator
num_lp_cf = [0, 2*zeta*wc_w, wc_w**2]
den_lp_cf = [1, 2*zeta*wc_w, wc_w**2]
num_hp_cf = [1, 0, 0]
den_hp_cf = [1, 2*zeta*wc_w, wc_w**2]

b_lp_cf, a_lp_cf = sp.signal.bilinear(num_lp_cf, den_lp_cf, fs)
b_hp_cf, a_hp_cf = sp.signal.bilinear(num_hp_cf, den_hp_cf, fs)

# Current FF1 filter
b_lp_f1, a_lp_f1 = sp.signal.butter(order, f_c*overlap, btype='low', fs=fs)
b_hp_f1, a_hp_f1 = sp.signal.butter(order, f_c/overlap, btype='high', fs=fs)

# S_ae transfer function for EEA calculation
num_Sae = [1, 0, 0]
den_Sae = [1, 2 * np.sqrt(2) * np.pi, 4 * np.pi**2]
S_ae = sp.signal.lti(num_Sae, den_Sae)

# Define frequency grids
f_h_list = [0.1, 0.5, 1, 5]   # ship movement frequencies
f_w_list = [0.5, 1, 2]  # wave frequencies

results = []

total_iterations = len(f_h_list) * len(f_w_list)
iteration = 0

randn = np.random.randn(len(t))

for f_h in f_h_list:
    for f_w in f_w_list:
        iteration += 1
        print(f"Processing {iteration}/{total_iterations}: f_h={f_h:.2f} Hz, f_w={f_w:.2f} Hz", end='\r')
        # Ship and wave motion
        h = 1 + 0.1 * np.sin(2 * np.pi * f_h * t)
        n = 0.2 * np.sin(2 * np.pi * f_w * t)

        # Accelerometer
        h_ddot = -0.4 * np.pi**2 * f_h**2 * np.sin(2 * np.pi * f_h * t)
        v_a = S_a * np.sqrt(fs) * randn
        y_a = h_ddot + b + v_a

        # Double integration
        y_a_i = sp.integrate.cumulative_trapezoid(y_a, x=t, initial=0)
        h_a = sp.integrate.cumulative_trapezoid(y_a_i, x=t, initial=0)

        # Height sensor
        y_h = h - n

        # FF1 filter
        y_h_lp_f1 = sp.signal.lfilter(b_lp_f1, a_lp_f1, y_h)
        h_a_hp_f1 = sp.signal.lfilter(b_hp_f1, a_hp_f1, h_a)
        h_f1 = y_h_lp_f1 + h_a_hp_f1

        # Complementary filter
        y_h_lp_cf = sp.signal.lfilter(b_lp_cf, a_lp_cf, y_h)
        h_a_hp_cf = sp.signal.lfilter(b_hp_cf, a_hp_cf, h_a)
        h_f2 = y_h_lp_cf + h_a_hp_cf

        # Mask to evaluate steady state
        mask = (t >= 30) & (t <= 100)
        
        # Mean
        mean_f1 = np.mean(h_f1[mask] - h[mask])
        mean_f2 = np.mean(h_f2[mask] - h[mask])

        # RMSE
        rmse_f1 = np.sqrt(np.mean((h_f1[mask] - h[mask])**2))
        rmse_f2 = np.sqrt(np.mean((h_f2[mask] - h[mask])**2))

        # EEA RMS
        _, eea_f1, _ = sp.signal.lsim(S_ae, U=(h_f1 - h), T=t)
        _, eea_f2, _ = sp.signal.lsim(S_ae, U=(h_f2 - h), T=t)
        eea_f1_rms = np.sqrt(np.mean(np.square(eea_f1[mask])))
        eea_f2_rms = np.sqrt(np.mean(np.square(eea_f2[mask])))

        results.append({
            'f_h': f_h,
            'f_w': f_w,
            'mean_f1': mean_f1,
            'mean_f2': mean_f2,
            'rmse_f1': rmse_f1,
            'rmse_f2': rmse_f2,
            'eea_f1': eea_f1_rms,
            'eea_f2': eea_f2_rms
        })

# Convert to structured array for easy display
import pandas as pd
df = pd.DataFrame(results)
print(df)


    f_h  f_w   mean_f1   mean_f2   rmse_f1   rmse_f2    eea_f1    eea_f2
0   0.1  0.5  0.000369  0.000330  0.052619  0.035390  0.011618  0.008582
1   0.1  1.0  0.000369  0.000330  0.025211  0.016308  0.008967  0.011523
2   0.1  2.0  0.000369  0.000330  0.022022  0.008002  0.003094  0.007738
3   0.5  0.5  0.000369  0.000330  0.046544  0.035390  0.011287  0.008582
4   0.5  1.0  0.000369  0.000330  0.024648  0.016308  0.010326  0.011523
5   0.5  2.0  0.000369  0.000330  0.021375  0.008002  0.005983  0.007738
6   1.0  0.5  0.000369  0.000330  0.049568  0.035390  0.014707  0.008582
7   1.0  1.0  0.000369  0.000330  0.016629  0.016308  0.011751  0.011523
8   1.0  2.0  0.000369  0.000330  0.013160  0.008002  0.009535  0.007738
9   5.0  0.5  0.000369  0.000329  0.047972  0.035391  0.011917  0.008582
10  5.0  1.0  0.000369  0.000329  0.012968  0.016308  0.009351  0.011523
11  5.0  2.0  0.000369  0.000329  0.004189  0.008002  0.004075  0.007738


In [48]:
# Make a copy to preserve original
df_c = df.copy()

# % improvement = (f1 - f2) / f1 * 100
#df_c['mean_%'] = (df['mean_f1'] - df['mean_f2']) / df['mean_f1'] * 100
df_c['rmse_%'] = (df['rmse_f2'] - df['rmse_f1']) / df['rmse_f1'] * 100
df_c['eea_%']  = (df['eea_f2'] - df['eea_f1']) / df['eea_f1'] * 100

# Optional: round for readability
df_c = df_c.round({'f_h': 2, 'f_w': 4, 'mean_f1': 3, 'mean_f2': 3, 'rmse_f1':4, 'rmse_f2':4, 'eea_f1':5, 'eea_f2':5, 
               'rmse_%':2, 'eea_%':2})

print(df_c)

    f_h  f_w  mean_f1  mean_f2  rmse_f1  rmse_f2   eea_f1   eea_f2  rmse_%  \
0   0.1  0.5      0.0      0.0   0.0526   0.0354  0.01162  0.00858  -32.74   
1   0.1  1.0      0.0      0.0   0.0252   0.0163  0.00897  0.01152  -35.31   
2   0.1  2.0      0.0      0.0   0.0220   0.0080  0.00309  0.00774  -63.66   
3   0.5  0.5      0.0      0.0   0.0465   0.0354  0.01129  0.00858  -23.96   
4   0.5  1.0      0.0      0.0   0.0246   0.0163  0.01033  0.01152  -33.83   
5   0.5  2.0      0.0      0.0   0.0214   0.0080  0.00598  0.00774  -62.56   
6   1.0  0.5      0.0      0.0   0.0496   0.0354  0.01471  0.00858  -28.60   
7   1.0  1.0      0.0      0.0   0.0166   0.0163  0.01175  0.01152   -1.93   
8   1.0  2.0      0.0      0.0   0.0132   0.0080  0.00953  0.00774  -39.20   
9   5.0  0.5      0.0      0.0   0.0480   0.0354  0.01192  0.00858  -26.23   
10  5.0  1.0      0.0      0.0   0.0130   0.0163  0.00935  0.01152   25.76   
11  5.0  2.0      0.0      0.0   0.0042   0.0080  0.00408  0.007

In [49]:
# Print in LaTeX table format
for _, row in df_c.iterrows():
    print(f"{row['f_h']:.2f} & {row['f_w']:.2f} & "
          f"{row['mean_f1']:.4f} & {row['mean_f2']:.4f} & "
          f"{row['rmse_f1']:.4f} & {row['rmse_f2']:.4f} & "
          f"{row['rmse_%']:.2f}\\% & "
          f"{row['eea_f1']:.4f} & {row['eea_f2']:.4f} & "
          f"{row['eea_%']:.2f}\\% \\\\ \\hline")

0.10 & 0.50 & 0.0000 & 0.0000 & 0.0526 & 0.0354 & -32.74\% & 0.0116 & 0.0086 & -26.13\% \\ \hline
0.10 & 1.00 & 0.0000 & 0.0000 & 0.0252 & 0.0163 & -35.31\% & 0.0090 & 0.0115 & 28.51\% \\ \hline
0.10 & 2.00 & 0.0000 & 0.0000 & 0.0220 & 0.0080 & -63.66\% & 0.0031 & 0.0077 & 150.09\% \\ \hline
0.50 & 0.50 & 0.0000 & 0.0000 & 0.0465 & 0.0354 & -23.96\% & 0.0113 & 0.0086 & -23.97\% \\ \hline
0.50 & 1.00 & 0.0000 & 0.0000 & 0.0246 & 0.0163 & -33.83\% & 0.0103 & 0.0115 & 11.59\% \\ \hline
0.50 & 2.00 & 0.0000 & 0.0000 & 0.0214 & 0.0080 & -62.56\% & 0.0060 & 0.0077 & 29.33\% \\ \hline
1.00 & 0.50 & 0.0000 & 0.0000 & 0.0496 & 0.0354 & -28.60\% & 0.0147 & 0.0086 & -41.65\% \\ \hline
1.00 & 1.00 & 0.0000 & 0.0000 & 0.0166 & 0.0163 & -1.93\% & 0.0118 & 0.0115 & -1.94\% \\ \hline
1.00 & 2.00 & 0.0000 & 0.0000 & 0.0132 & 0.0080 & -39.20\% & 0.0095 & 0.0077 & -18.85\% \\ \hline
5.00 & 0.50 & 0.0000 & 0.0000 & 0.0480 & 0.0354 & -26.23\% & 0.0119 & 0.0086 & -27.98\% \\ \hline
5.00 & 1.00 & 0.0000 & 0.